In [62]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

import numpy as np
import pandas as pd


In [3]:
engine = create_engine("sqlite:///datasets/belly_button_biodiversity.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
otu = Base.classes.otu
samples = Base.classes.samples
samples_meta = Base.classes.samples_metadata

# Create our session (link) from Python to the DB
session = Session(engine)


In [5]:
inspector = inspect(engine)

In [6]:
inspector.get_table_names()

['otu', 'samples', 'samples_metadata']

In [7]:
columns = inspector.get_columns('otu')
for column in columns:
    print(column["name"], column["type"])

otu_id INTEGER
lowest_taxonomic_unit_found TEXT


In [44]:
columns = inspector.get_columns('samples_metadata')
for column in columns:
    print(column["name"], column["type"])

SAMPLEID INTEGER
EVENT TEXT
ETHNICITY TEXT
GENDER TEXT
AGE INTEGER
WFREQ INTEGER
BBTYPE TEXT
LOCATION TEXT
COUNTRY012 TEXT
ZIP012 INTEGER
COUNTRY1319 TEXT
ZIP1319 INTEGER
DOG TEXT
CAT TEXT
IMPSURFACE013 INTEGER
NPP013 FLOAT
MMAXTEMP013 FLOAT
PFC013 FLOAT
IMPSURFACE1319 INTEGER
NPP1319 FLOAT
MMAXTEMP1319 FLOAT
PFC1319 FLOAT


In [27]:
#app.route('/names')
columns = inspector.get_columns('samples')
names = []
for column in columns:
    names.append(column['name'])
del names[0]
#return jsonify(names)

In [106]:
#app.route('/otu')
results = session.query(otu.otu_id, otu.lowest_taxonomic_unit_found).all()
#descriptions = [result[0],result[1] for result in results]
#descriptions
dftest = pd.DataFrame(results)
dftest = dftest.head()
dftest

#return jsonify(descriptions)

,otu_id,lowest_taxonomic_unit_found
0,1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...
1,2,Archaea;Euryarchaeota;Halobacteria;Halobacteri...
2,3,Archaea;Euryarchaeota;Halobacteria;Halobacteri...
3,4,Archaea;Euryarchaeota;Methanobacteria;Methanob...
4,5,Archaea;Euryarchaeota;Methanobacteria;Methanob...


In [119]:
otu_count = len(dftest.index)
otu_info = {dftest['otu_id'][0], dftest['lowest_taxonomic_unit_found'][0]}
otu_info = []

for x in range(otu_count):
    otu_info.append({dftest['otu_id'][x]: dftest['lowest_taxonomic_unit_found'][x]})
otu_info

[{1: 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus'},
 {2: 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus'},
 {3: 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Natronorubrum'},
 {4: 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter'},
 {5: 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter'}]

In [ ]:
#@app.route('/metadata/<sample>')
def sample_metadata(sample):
    sample_id = sample.replace('BB_','')
    result = session.query(samples_meta).filter(samples_meta.SAMPLEID == sample_id).first()
    metadata = {
        'AGE':result.AGE,
        'BBTYPE':result.BBTYPE,
        'ETHNICITY':result.ETHNICITY,
        'GENDER':result.GENDER,
        'LOCATION':result.LOCATION,
        'SAMPLEID':result.SAMPLEID
    }
    return jsonify(metadata)

In [ ]:
#app.route('/wfreq/<sample>')
def washing_frequency(sample):
    sample_id = sample.replace('BB_','')
    result = session.query(samples_meta).filter(samples_meta.SAMPLEID == sample_id).first()
    return result.WFREQ

In [ ]:
#app.route('/samples/<sample>')
def sample_count(sample):
    

In [69]:
sample = 'BB_940'
resulting = session.query(samples.otu_id, getattr(samples, sample)).all()
df = pd.DataFrame(resulting)
df = df.set_index('otu_id')


In [82]:
test = df.sort_values(by=[sample], ascending=False).head(10).reset_index()
test

,otu_id,BB_940
0,1167,163
1,2859,126
2,482,113
3,2264,78
4,41,71
5,1189,51
6,352,50
7,189,47
8,2318,40
9,1977,40


In [96]:
ids = tuple(test['otu_id'].values.tolist())
vals = tuple(test[sample].values.tolist())

In [97]:
len(ids)

10

In [98]:
len(vals)

10

In [99]:
vals

(163, 126, 113, 78, 71, 51, 50, 47, 40, 40)

In [100]:
ids

(1167, 2859, 482, 2264, 41, 1189, 352, 189, 2318, 1977)

In [101]:
sample_counts = {'otu_ids':ids,'sample_values':vals}

In [102]:
sample_counts

{'otu_ids': (1167, 2859, 482, 2264, 41, 1189, 352, 189, 2318, 1977),
 'sample_values': (163, 126, 113, 78, 71, 51, 50, 47, 40, 40)}